In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import re

observations = pd.read_csv('observations.csv')
species_info = pd.read_csv('species_info.csv')

In [2]:
observations.describe(include='all')

,scientific_name,park_name,observations
count,23296,23296,23296.000000
unique,5541,4,NaN
top,Myotis lucifugus,Great Smoky Mountains National Park,NaN
freq,12,5824,NaN
mean,NaN,NaN,142.287904
std,NaN,NaN,69.890532
min,NaN,NaN,9.000000
25%,NaN,NaN,86.000000
50%,NaN,NaN,124.000000
75%,NaN,NaN,195.000000


In [3]:
observations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23296 entries, 0 to 23295
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   scientific_name  23296 non-null  object
 1   park_name        23296 non-null  object
 2   observations     23296 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 546.1+ KB


In [4]:
# dupes?
observations.duplicated().any()

True

In [5]:
observations.drop_duplicates(inplace=True)

In [6]:
species_info.describe(include='all')

,category,scientific_name,common_names,conservation_status
count,5824,5824,5824,191
unique,7,5541,5504,4
top,Vascular Plant,Castor canadensis,Brachythecium Moss,Species of Concern
freq,4470,3,7,161


In [7]:
species_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5824 entries, 0 to 5823
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   category             5824 non-null   object
 1   scientific_name      5824 non-null   object
 2   common_names         5824 non-null   object
 3   conservation_status  191 non-null    object
dtypes: object(4)
memory usage: 182.1+ KB


In [8]:
# conservation status is full of nans
species_info.conservation_status.unique()

array([nan, 'Species of Concern', 'Endangered', 'Threatened',
       'In Recovery'], dtype=object)

In [9]:
# based on this i am making the assumption that nan values in this column represent that the associated species is not threatened, so i am updating
# the nans to reflect that and to make my life easier going forward.
# https://en.wikipedia.org/wiki/United_States_Fish_and_Wildlife_Service_list_of_endangered_mammals_and_birds
# wikipedia says the language is endangered or threatened, so i will just fill nans with 'Not Threatened'.
species_info.conservation_status = species_info.conservation_status.fillna('Not Threatened')

In [10]:
# checking to see if there are any duplicate rows
species_info.duplicated().any()

False

In [11]:
# since both frames have a scientific_name column with the exact same number of uniques, i want to confirm these uniques are identical

unique_observations_name = set(observations['scientific_name'].unique())

unique_species_info_name = set(species_info['scientific_name'].unique())

print(unique_observations_name == unique_species_info_name)

True


In [12]:
# species info having multiple instances of scientific names is perplexing to me, so i want to figure out why it is the way it is

value_counts = species_info.scientific_name.value_counts()
value_counts_filtered = value_counts[value_counts > 1].index
multiple_entries_df = species_info[species_info['scientific_name'].isin(value_counts_filtered)]
multiple_entries_df = multiple_entries_df.sort_values(by='scientific_name')
# multiple_entries_df.head(30)

In [13]:
# based on a visual inspection of these apparent duplicate rows i am guessing the common names are the differentiating factor, so i am going to combine/concatenate
# the common_name across rows that share a scientific name so long as conservation status is also identical across rows that share a scientific name

# check for dupes, if true my guesstimation was accurate, or at least partially accurate
columns_to_check = ['category', 'scientific_name', 'conservation_status']
duplicates = species_info.duplicated(subset=columns_to_check, keep=False).any()
print(duplicates)

True


In [14]:
species_info = species_info.groupby(columns_to_check).agg({'common_names': ', '.join}).reset_index()

In [15]:
species_info.describe(include='all')

,category,scientific_name,conservation_status,common_names
count,5543,5543,5543,5543
unique,7,5541,5,5238
top,Vascular Plant,Oncorhynchus mykiss,Not Threatened,Brachythecium Moss
freq,4262,2,5363,7


In [16]:
# some duplicates persist, so lets take a look at what they are this time using the same thing as last time
value_counts = species_info.scientific_name.value_counts()
value_counts_filtered = value_counts[value_counts > 1].index
multiple_entries_df = species_info[species_info['scientific_name'].isin(value_counts_filtered)]
multiple_entries_df = multiple_entries_df.sort_values(by='scientific_name')
multiple_entries_df.head()

,category,scientific_name,conservation_status,common_names
705,Mammal,Canis lupus,Endangered,"Gray Wolf, Gray Wolf, Wolf"
706,Mammal,Canis lupus,In Recovery,"Gray Wolf, Wolf"
664,Fish,Oncorhynchus mykiss,Not Threatened,Rainbow Trout
665,Fish,Oncorhynchus mykiss,Threatened,Rainbow Trout


In [17]:
# it appears that these entries were updated, and given that the indexes are in sequential order i will operate on the assumption that a larger index indicates an update to the original values
species_info = species_info.drop([664, 705])

In [18]:
species_info.describe(include='all')

,category,scientific_name,conservation_status,common_names
count,5541,5541,5541,5541
unique,7,5541,5,5237
top,Vascular Plant,Acris crepitans crepitans,Not Threatened,Brachythecium Moss
freq,4262,1,5362,7


In [19]:
# now i want to go back and clean up common_names that had some concatenating. i want to preserve all unique descriptors, so i will split by ',' and ' '
# then create a set of the individual words to ensure no duplicates, then strip any parentheses, then recombine into single strings for easy searching
species_info.common_names = (
    species_info.common_names
    .str.split(r'[, ]')
    .apply(set)
    .apply(list)
    .apply(lambda lst: [item.strip("()") for item in lst])
    .str.join(' ')
)

In [20]:
# now the only thing i can think that needs addressing is confirming conservation_status is ordinal, as the rest of the categorical data here is decidedly nominal.
# based on 
species_info.conservation_status = pd.Categorical(
    species_info.conservation_status, 
    ['Not Threatened', 'In Recovery', 'Species of Concern', 'Threatened', 'Endangered'],
    ordered=True,
)

In [21]:
# now that common_names is cleaned up i want to see the frequency of each unique word, the goal being to use that to identify umbrella terms used within
# each 'category', so that i can compare this data with greater granularity
categories = species_info.category.unique().tolist()

# empty dictionary to store arrays in
word_count_by_category = {}

for value in categories:
    species_info_filtered = species_info[species_info.category == value]
    # combine all strings in common_names
    all_common_names = ' '.join(species_info_filtered['common_names'])
    # split full string into list by ' ' (default)
    all_common_names_words = all_common_names.split()
    # create a series with the list of words as input, then get the counts of each value
    word_counts = pd.Series(all_common_names_words).value_counts()
    # store list in word_count_by_category dictionary
    word_count_by_category[value] = word_counts

In [22]:
print(categories)

['Amphibian', 'Bird', 'Fish', 'Mammal', 'Nonvascular Plant', 'Reptile', 'Vascular Plant']


In [23]:
# word_count_by_category['Amphibian'].head(60)

# common_names is much cleaner but there still appear to be oddities like 'Salamander' vs 'Salamanders'

# i dont want to rely on my own eyes to ensure there are no typos or accidental trailing letters, so i will copy/paste the most frequent unique words collected above
# into my own little lists of common words to use as identifiers/subcategories, and then i will remove all adjectives that are not classifiers (this ended up taking
# a fair amount of time since i lack anything resembling expertise in this field, so i used wikipedia for anything i did not understand, which was significant)

# there are so so so many unique words in vascular plant, i will need to figure out how to keep the list trim. this may take a few editing passes. jupyterlab only
# lets me print 60 rows at a time, either by default or by my own confused selections in the settings menu. regardless of *why* it is the way it is, i opted to just
# call the indeces of the 'Vascular Plant' series stored in the dictionary a couple cells up

# word_count_by_category['Vascular Plant'].iloc[240:300]

In [24]:
amphibian_terms = [
    'Salamander',
    'Frog',
    'Toad',
    'Salamanders',
    'Treefrog',
    'Ensatina',
    'Bullfrog',
    'Newt',
]

bird_terms = [
    'Warbler',
    'Sparrow',
    'Flycatcher',
    'Owl',
    'Woodpecker',
    'Sandpiper',
    'Wren',
    'Gull',
    'Hummingbird',
    'Vireo',
    'Hawk',
    'Tern',
    'Finch',
    'Grosbeak',
    'Swallow',
    'Goose',
    'Duck',
    'Grebe',
    'Thrush',
    'Grouse',
    'Heron',
    'Scoter',
    'Oriole',
    'Junco',
    'Egret',
    'Swift',
    'Pine',
    'Goldfinch',
    'Blackbird',
    'Towhee',
    'Chickadee',
    'Bunting',
    'Sapsucker',
    'Jay',
    'Screech-Owl',
    'Merganser',
]

fish_terms = [
    'Shiner',
    'Darter',
    'Trout',
    'Sucker',
    'Dace',
    'Chub',
    'Bass',
    'Sunfish',
    'Redhorse',
    'Sculpin',
    'Catfish',
    'Lamprey',
    'Longnose',
    'Minnow',
    'Mosquitofish',
    'Blacktail',
    'Whitetail',
    'Stoneroller',
]

mammal_terms = [
    'Bat',
    'Mouse',
    'Shrew',
    'Myotis',
    'Chipmunk',
    'Squirrel',
    'Vole',
    'Woodrat',
    'Fox',
    'Mole',
    'Rat',
    'Weasel',
    'Lion',
    'Mink',
    'Skunk',
    'Deermouse',
    'Deer',
    'Gopher',
    'Bear',
    'Sheep',
    'Fisher',
    'Otter',
    'Lemming',
    'Jackrabbit',
    'Rabbit',
    'Hare',
]

# wikipedia is a godsend: non-vascular plants include Bryophytes (mosses, liverwort, and hornwort) and Algae.
nonvascular_terms = [
    'Moss',
    'Sphagnum',         # subset of moss
    'Dicranum',         # subset of moss
    'Bryum',            # subset of moss
    'Brachythecium',    # subset of moss
    'Hypnum',           # subset of moss
    'Fissidens',        # subset of moss
    'Polytrichum',      # subset of moss
    'Pohlia',           # subset of moss
    'Cephalozia',       # subset of liverwort
    'Entodon',          # subset of moss
    'Fontinalis',       # subset of moss
    'Calypogeja',       # subset of liverwort
    'Atrichum',         # subset of moss
    'Hygrohypnum',      # subset of moss
    'Philonotis',       # subset of moss
    'Anomodon',         # subset of moss
    'Anastrophyllum',   # subset of liverwort
    'Plagiomnium',      # subset of moss
    'Ulota',            # subset of moss
    'Calcareous',       # subset of moss - calcareous (adjective): "mostly or partly composed of calcium carbonate"
    'Bazzania',         # subset of liverwort
    'Orthotrichum',     # subset of moss
    'Ditrichum',        # subset of moss
    'Campylium',        # subset of moss
    'Schistidium',      # subset of moss - this term and all terms below appear 3 or fewer times
    'Dicranella',       # subset of moss
    'Sematophyllum',    # subset of moss
    'Rhizomnium',       # subset of moss
    'Funaria',          # subset of moss
    'Plagiothecium',    # subset of moss
    'Thuidium',         # subset of moss
    'Pylaisiella',      # subset of algae
    'Racomitrium',      # subset of moss
    'Campylopus',       # subset of moss
    'Didymodon',        # subset of moss
    'Cryphaea',         # subset of moss
    'Dicranodontium',   # subset of moss
    'Climacium',        # subset of moss
    'Zygodon',          # subset of moss
    'Pseudotaxiphyllum',# subset of moss
    'Diphyscium',       # subset of moss
    'Leucodon',         # subset of moss
    'Leucobryum',       # subset of moss
    'Leskea',           # subset of moss
    'Leptodontium',     # subset of moss
    'Neckera',          # subset of moss
]

reptile_terms = [
'Snake',
'Turtle',
'Lizard',
'Skink',
'Kingsnake',      # subset of snake
'Garter',         # subset of snake
'Racer',          # subset of snake
'Rattlesnake',    # subset of snake
'Worm',
'Alligator',
]

# this is going to be a lot to filter. i wish i knew this information already so i didnt have to look it all up.
# i have cut myself off at a minumum of 10 instances of each unique word. otherwise there would be too many to include.
vascular_sub = [
    'Sedge',
    'Grass',
    'Rush',
    'Fern',
    'Aster',
    'Willow',
    'Buttercup',
    'Fleabane',
    'Goldenrod',
    'Pine',
    'Buckwheat',
    'Cinquefoil',
    'Daisy',
    'Bluegrass',
    'Monkeyflower',
    'Wood',
    'Panic',
    'Brome',
    'Fescue',
    'Rockcress',
    'Pondweed',
    'Clover',
    'Phacelia',
    'Milkvetch',
    'Thistle',
    'Paintbrush',
    'Oak',
    'Lupine',
    'Sunflower',
    'Rose',
    'Bedstraw',
    'Saxifrage',
    'Orchid',
    'Lily',
    'Groundsel',
    'Ragwort',
    'Chickweed',
    'Knotweed',
    'Phlox',
    'Willowherb',
    'Pussytoes',
    'Bulrush',
    'Draba',
    'Penstemon',
    'Mustard',
    'Goosefoot',
    'Bentgrass',
    'Smallflower',
    'Downy',
    'Snakeroot',
    'Onion',
    'Flatsedge',
    'Plantain',
    'Willowweed',
    'Sagebrush',
    'Gilia',
    'Hawkweed',
    'Speedwell',
    'Honeysuckle',
    'Skullcap',
    'Campion',
    'Cryptantha',
    'Spikerush',
    'Cutleaf',
    'Gentian',
    'Dandelion',
    'Needlegrass',
    'Stonecrop',
    'Milkweed',
    'Muhly',
    'Catchfly',
    'Clubmoss',
    'Beardtongue',
    'Starwort',
    'Arnica',
    'Tick-Trefoil',
    'Currant',
    'Grape',
    'Foxtail',
    'Lettuce',
    'Primrose',
    'Larkspur',
    'Pepperweed',
    'Mistletoe',
    'Maple',
    'Meadowrue',
    'Hawthorn',
    'Sandwort',
    'Agoseris',
    'Alumroot',
    'Corn',
    'Clarkia',
    'Spurge',
    'Blackberry',
    'Woodrush',
    'Lanceleaf',
    'Hawksbeard',
    'Trillium',
    'Rye',
    'Avens',
    'Oatgrass',
    'Azalea',
    'Pea',
    'Alkali',
    'Sumac',
]

In [25]:
# now that i have these lists of terms pruned down and ready for copy/pasting i can create a function that will narrow the dataframe by common_names filter terms
def filter_by_common_names(df, filter_list, all_words=True):
    # creating regex pattern from filter_list words
    pattern = r'\b(?:' + '|'.join(re.escape(word) for word in filter_list) + r')\w*\b'
    regex = re.compile(pattern, re.IGNORECASE)

    # checking to see how to filter by filter_list: True = rows must include all filter words, False = rows must contain any filter words
    if all_words:
        mask = df['common_names'].apply(lambda x: all(regex.search(word) for word in x.split()))
    else:
        mask = df['common_names'].apply(lambda x: any(regex.search(word) for word in x.split()))
    # return a whole dataframe so it can be saved as an appropriate variable
    return df[mask]

In [26]:
# alright i have my cleaned dataframes, my lists of preapproved search terms, the search function for the preapproved search terms, i think it is time to
# combine the dataframes and begin exploring the relationships between the data.

# keeping new dataframe name simple, taking a copy of the data in each row of species_info and appending to each row of observations where scientific_name overlaps
df = pd.merge(observations, species_info, on='scientific_name', how='left')